In [1]:
from collections import deque, defaultdict
from itertools import combinations
from tqdm import tqdm

In [2]:
filename = "sample.txt"
# filename = "input.txt"
with open(filename, encoding="utf-8") as f:
    data = f.read()

lines = data.strip().split("\n")

https://adventofcode.com/2024/day/20

In [3]:
track = set()
walls = set()
for y, line in enumerate(lines):
    for x, c in enumerate(line):
        pos = complex(x, y)
        match c:
            case '#':
                walls.add(pos)
            case '.':
                track.add(pos)
            case 'S':
                track.add(pos)
                start_pos = pos
            case 'E':
                track.add(pos)
                end_pos = pos

In [4]:
## Part 1
# There's only one path from S to E, cheats move onto a wall, then back onto a path
# Find the path. For each wall next to the path, pair up their neighbours (in sorted-path order)
# Time saved is based on the diff in index of the two path-nodes of the pair
def adjacent(pos):
    return [pos + step for step in (1, 1j, -1, -1j)]

def get_path(parents: dict, end: complex):
    out = [end]
    pos = parents.get(end)
    while pos is not None:
        out.append(pos)
        pos = parents.get(pos)
    return out[::-1]

def bfs(grid, start, end):
    # Reuse BFS from d18 to get the path in order
    parent = {start: None}
    frontier = deque([start])
    while frontier:
        # Note: since all steps are the same size, BFS is guaranteed to find the shortest path if we explore frontier FIFO
        pos = frontier.popleft()  
        if pos == end:
            return get_path(parent, pos)
        # Mark all reachable neighbours of pos as seen
        for n in adjacent(pos):
            if n in parent:
                # Already seen
                continue
            if n in grid:
                parent[n] = pos
                frontier.append(n)
    print(f"BFS couldn't find path from {start} to {end}. {parent=}")

path = bfs(track, start_pos, end_pos)
len(path)  # Note: includes start position

85

In [5]:
# For each wall next to the path, find the neighbours that can cut through it
# Pair those up and find the shortcut saving
wall_neighbours = defaultdict(list)
for pos in path:
    for n in adjacent(pos):
        if n in walls:
            wall_neighbours[n].append(pos)

# wall_neighbours

In [6]:
shortcuts = []
for wall, ns in wall_neighbours.items():
    for a, b in combinations(ns, 2):
        ind_a, ind_b = path.index(a), path.index(b)
        # Because we added neighbours in path-order, we're guaranteed that ind_b > ind_a. But abs(diff) anyway, just to be sure
        time_saved = abs(ind_b - ind_a) - 2  # Shortcuts always cost 2 to execute
        shortcuts.append((time_saved, wall, a, b))
        # print(sorted([a, b], key=lambda x: path.index(x)))
# shortcuts

In [7]:
# # How many cheats are there, grouped by time saved?
# from collections import Counter
# c = Counter(x[0] for x in shortcuts)
# print(sorted(c.items()))

In [8]:
# How many cheats would save at least 100 picoseconds?
quick_cheats = [x[0] for x in shortcuts if x[0] >= 100]
len(quick_cheats)

0

In [9]:
## Part 2
# 20-step max cheats
# Cheats are still uniquely identified by (start, end) positions, so path taken within the cheat doesn't matter
# Cost of cheat = taxicab distance(start, end)
# Option 1 (probably too many loops):
#  For every path, consider every future pos of the path where distance <= 20
#  if time-saving >= 50, count it
def taxicab_dist(p1, p2):
    d = p1 - p2
    return abs(d.real) + abs(d.imag)

In [10]:
# Brute-force attempt that tries all 45,167,760 combinations of A,B. Takes ~9mins
# There's definitely a better way that cuts down the search space
# - iterating over path, path[i+2:] to cut out all 0 and 1-step shortcuts
# -- ^ note from reddit: To save >= 100 steps, B must be from path[i+100:]
# - sweep-line algorithm? Anything >= 20x or >=20y away shouldn't be paired 
# !! Use dict {pos: index} instead of list.index() -> takes ~
distances = {pos: i for i, pos in enumerate(path)}

p2_cheat_count = 0
for a, b in tqdm(combinations(path, 2)):
    cheat_cost = taxicab_dist(a, b)
    # cheat_cost = abs(b.real - a.real) + abs(b.imag - a.imag)
    if cheat_cost > 20:
        continue
    # ind_a, ind_b = path.index(a), path.index(b)  # This is linear to the list-length! VERY SLOW!
    # time_saved = abs(ind_b - ind_a) - cheat_cost
    time_saved = abs(distances[b] - distances[a]) - cheat_cost

    if time_saved >= 100:
        p2_cheat_count += 1

p2_cheat_count

3570it [00:00, 426843.37it/s]


0

In [11]:
# Speed-up attempt: manually iterate over pairs -> 32sec
p2_cheat_count = 0
for i in tqdm(range(len(path))):
    a = path[i]
    for b in path[i+101:]:
        cheat_cost = taxicab_dist(a, b)
        if cheat_cost > 20:
            continue
        time_saved = abs(distances[b] - distances[a]) - cheat_cost

        if time_saved >= 100:
            p2_cheat_count += 1

p2_cheat_count

100%|██████████| 85/85 [00:00<00:00, 129642.12it/s]


0